# Finetunnning LLM LawBot

## Install relevant packages

In [1]:
%%capture

!pip install unsloth
!pip install bitsandbytes
!pip install unsloth_zoo
!pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

## Import all relevant packages throughout this walkthrough

In [2]:
# Modules for fine-tuning
from unsloth import FastLanguageModel
import torch # Import PyTorch
from trl import SFTTrainer # Trainer for supervised fine-tuning (SFT)
from unsloth import is_bfloat16_supported # Checks if the hardware supports bfloat16 precision
# Hugging Face modules
from huggingface_hub import login # Lets you login to API
from transformers import TrainingArguments # Defines training hyperparameters
from datasets import load_dataset # Lets you load fine-tuning datasets
# Import weights and biases
import wandb
# Import kaggle secrets
from kaggle_secrets import UserSecretsClient

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-05-22 00:33:55.573288: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747874035.806495      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747874035.872727      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!


## Create API keys and login to Hugging Face and Weights and Biases

In [3]:
user_secrets = UserSecretsClient()
hugging_face_token = user_secrets.get_secret("HF_TOKEN_DEEPSEEK")
wnb_token = user_secrets.get_secret("wnb_token")

login(hugging_face_token)

wandb.login(key=wnb_token)
run = wandb.init(
    project='Fine-tune-DeepSeek-R1-Distill-Llama-8B on LawBot', 
    job_type="training", 
    anonymous="allow"
)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: lucky-sntso (lucky-santoso) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Tracking run with wandb version 0.19.6
wandb: Run data is saved locally in /kaggle/working/wandb/run-20250522_003417-1sqfsyu6
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run misunderstood-wind-4
wandb: ⭐️ View project at https://wandb.ai/lucky-santoso/Fine-tune-DeepSeek-R1-Distill-Llama-8B%20on%20LawBot?apiKey=15cb28260808b8ec8d2f49df0f2f1133f0c66080
wandb: 🚀 View run at https://wandb.ai/lucky-santoso/Fine-tune-DeepSeek-R1-Distill-Llama-8B%

## Loading Model and the Tokenizer

In [4]:
max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/DeepSeek-R1-Distill-Llama-8B",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    token=hugging_face_token,
)

==((====))==  Unsloth 2025.5.7: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla P100-PCIE-16GB. Num GPUs = 1. Max memory: 15.888 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 6.0. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

## Testing Model on a law use-case before fine-tuning

In [5]:
prompt_style = """
Di bawah ini adalah instruksi yang menjelaskan tugas, dipasangkan dengan input yang memberikan konteks lebih lanjut.
Tuliskan respons yang menyelesaikan permintaan dengan tepat.
Sebelum menjawab, pikirkan dengan cermat pertanyaan tersebut dan buatlah rangkaian pemikiran langkah demi langkah untuk memastikan respons yang logis dan akurat.

### Instruksi:
Anda adalah seorang ahli hukum dengan pengetahuan tingkat lanjut dalam penalaran hukum, analisis kasus, dan penyusunan dokumen hukum. 
Jawablah pertanyaan hukum berikut ini dengan tepat, berdasarkan peraturan perundang-undangan yang berlaku dan preseden hukum yang relevan.

### Pertanyaan:
{}

### Jawaban:
<think>
{}
"""

### Running inference on the model


In [6]:
question = """Apa arti dari “berada di bawah Presiden” dalam konteks TNI?"""

FastLanguageModel.for_inference(model)

inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)

response = tokenizer.batch_decode(outputs)

print(response[0].split("### Response:")[0])

<｜begin▁of▁sentence｜>
Di bawah ini adalah instruksi yang menjelaskan tugas, dipasangkan dengan input yang memberikan konteks lebih lanjut.
Tuliskan respons yang menyelesaikan permintaan dengan tepat.
Sebelum menjawab, pikirkan dengan cermat pertanyaan tersebut dan buatlah rangkaian pemikiran langkah demi langkah untuk memastikan respons yang logis dan akurat.

### Instruksi:
Anda adalah seorang ahli hukum dengan pengetahuan tingkat lanjut dalam penalaran hukum, analisis kasus, dan penyusunan dokumen hukum. 
Jawablah pertanyaan hukum berikut ini dengan tepat, berdasarkan peraturan perundang-undangan yang berlaku dan preseden hukum yang relevan.

### Pertanyaan:
Apa arti dari “berada di bawah Presiden” dalam konteks TNI?

### Jawaban:
<think>

"Berada di bawah Presiden" dalam konteks TNI dapat memiliki beberapa makna berdasarkan konteks tertentu. Pertama, jika berbicara tentang hierarki TNI, "berada di bawah Presiden" dapat merujuk pada pengelompokan personel TNI yang langsung berada di 

## Fine-tuning step by step

## Step 1 — Update the system prompt 

In [7]:
train_prompt_style = """
Di bawah ini adalah instruksi yang menjelaskan tugas, dipasangkan dengan input yang memberikan konteks lebih lanjut.
Tuliskan respons yang menyelesaikan permintaan dengan tepat.
Sebelum menjawab, pikirkan dengan cermat pertanyaan tersebut dan buatlah rangkaian pemikiran langkah demi langkah untuk memastikan respons yang logis dan akurat.

### Instruksi:
Anda adalah seorang ahli hukum dengan pengetahuan tingkat lanjut dalam penalaran hukum, analisis kasus, dan penyusunan dokumen hukum. 
Jawablah pertanyaan hukum berikut ini dengan tepat, berdasarkan peraturan perundang-undangan yang berlaku dan preseden hukum yang relevan.


### Pertanyaan:
{}

### Jawaban:
<think>
{}
</think>
{}
"""


## Step 2 — Download the fine-tuning dataset and format it for fine-tuning

In [8]:
import pandas as pd
df = pd.read_csv("/kaggle/input/dataset-law-chatbot/dataset.csv")
df.head()

,Context,Question,Answer
0,Pasal 3 ayat (2) sebelum perubahan menyatakan:...,Apa inti pengaturan Pasal 3 ayat (2) sebelum d...,"Intinya, semua urusan kebijakan, strategi pert..."
1,Pasal 3 ayat (2) sebelum perubahan menempatkan...,Bagaimana hubungan koordinatif TNI dengan Depa...,TNI wajib menyelaraskan kebijakan dan administ...
2,Istilah yang digunakan dalam ketentuan lama ad...,"Mengapa istilah ""Departemen Pertahanan"" pentin...",Karena pada masa itu nomenklatur resmi lembaga...
3,"Pada periode sebelum perubahan, fokus koordina...",Adakah penekanan khusus terkait perencanaan st...,Tidak ada; versi lama bersifat lebih umum dan ...
4,Ketentuan lama belum menyinggung secara ekspli...,Apakah ruang lingkup koordinasi Departemen Per...,"Belum, sehingga interpretasinya bisa sangat lu..."


In [9]:
EOS_TOKEN = tokenizer.eos_token  
EOS_TOKEN

'<｜end▁of▁sentence｜>'

In [10]:
def formatting_prompts_func(examples):
    inputs = examples["Question"]
    cots = examples["Context"]
    outputs = examples["Answer"]
    
    texts = []
    
    for input, cot, output in zip(inputs, cots, outputs):
        text = train_prompt_style.format(input, cot, output) + EOS_TOKEN
        texts.append(text)

    return {
        "text": texts,
    }

In [11]:
from datasets import Dataset
from datasets import concatenate_datasets

dataset = Dataset.from_pandas(df)

split_dataset = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset_raw = split_dataset["train"]
eval_dataset_raw = split_dataset["test"]

train_dataset = train_dataset_raw.map(formatting_prompts_func, batched=True)
eval_dataset = eval_dataset_raw.map(formatting_prompts_func, batched=True)

# Contoh preview
print(train_dataset["text"][0])

Map:   0%|          | 0/1750 [00:00<?, ? examples/s]

Map:   0%|          | 0/438 [00:00<?, ? examples/s]


Di bawah ini adalah instruksi yang menjelaskan tugas, dipasangkan dengan input yang memberikan konteks lebih lanjut.
Tuliskan respons yang menyelesaikan permintaan dengan tepat.
Sebelum menjawab, pikirkan dengan cermat pertanyaan tersebut dan buatlah rangkaian pemikiran langkah demi langkah untuk memastikan respons yang logis dan akurat.

### Instruksi:
Anda adalah seorang ahli hukum dengan pengetahuan tingkat lanjut dalam penalaran hukum, analisis kasus, dan penyusunan dokumen hukum. 
Jawablah pertanyaan hukum berikut ini dengan tepat, berdasarkan peraturan perundang-undangan yang berlaku dan preseden hukum yang relevan.


### Pertanyaan:
Siapa yang berwenang mengatur mekanisme kenaikan pangkat TNI?

### Jawaban:
<think>
Setiap prajurit berhak mendapat kenaikan pangkat/jabatan berdasarkan prestasi sesuai pola karier dan kebutuhan TNI.
</think>
Panglima TNI melalui keputusan resmi.
<｜end▁of▁sentence｜>


## Step 3 — Setting up the model using LoRA

In [12]:
model_lora_lawbot = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

Unsloth 2025.5.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [13]:
trainer_lawbot = SFTTrainer(
    model=model_lora_lawbot,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        num_train_epochs=1,
        warmup_steps=5,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=100,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        eval_strategy="steps",
        eval_steps=100,
        save_strategy="no"
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/1750 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/438 [00:00<?, ? examples/s]

## Step 4 — Model training! 

In [14]:
# Start the fine-tuning process
trainer_stats = trainer_lawbot.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,750 | Num Epochs = 1 | Total steps = 218
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
100,0.542900,0.292611
200,0.253400,0.246980


Unsloth: Not an error, but LlamaForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


In [15]:
# Save the fine-tuned model
wandb.finish()

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:               eval/loss █▁
wandb:            eval/runtime █▁
wandb: eval/samples_per_second ▁▁
wandb:   eval/steps_per_second ▁▁
wandb:             train/epoch ▁▁▇▇█
wandb:       train/global_step ▁▁▇▇█
wandb:         train/grad_norm █▁
wandb:     train/learning_rate █▁
wandb:              train/loss █▁
wandb: 
wandb: Run summary:
wandb:                eval/loss 0.24698
wandb:             eval/runtime 412.3912
wandb:  eval/samples_per_second 1.062
wandb:    eval/steps_per_second 0.531
wandb:               total_flos 2.881062030262272e+16
wandb:              train/epoch 0.99657
wandb:        train/global_step 218
wandb:          train/grad_norm 0.40169
wandb:      train/learning_rate 2e-05
wandb:               train/loss 0.2534
wandb:               train_loss 0.38581
wandb:            train_runtime 5544.4577
wandb: train_samples_per_second 0.316
wandb:   train_steps_

## Step 5 — Run model inference after fine-tuning

In [16]:
question = """Apa arti dari “berada di bawah Presiden” dalam konteks TNI?"""

FastLanguageModel.for_inference(model_lora_lawbot)

inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = model_lora_lawbot.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)

response = tokenizer.batch_decode(outputs)

print(response[0].split("### Response:")[0])


<｜begin▁of▁sentence｜>
Di bawah ini adalah instruksi yang menjelaskan tugas, dipasangkan dengan input yang memberikan konteks lebih lanjut.
Tuliskan respons yang menyelesaikan permintaan dengan tepat.
Sebelum menjawab, pikirkan dengan cermat pertanyaan tersebut dan buatlah rangkaian pemikiran langkah demi langkah untuk memastikan respons yang logis dan akurat.

### Instruksi:
Anda adalah seorang ahli hukum dengan pengetahuan tingkat lanjut dalam penalaran hukum, analisis kasus, dan penyusunan dokumen hukum. 
Jawablah pertanyaan hukum berikut ini dengan tepat, berdasarkan peraturan perundang-undangan yang berlaku dan preseden hukum yang relevan.

### Pertanyaan:
Apa arti dari “berada di bawah Presiden” dalam konteks TNI?

### Jawaban:
<think>

TNI berada di bawah Presiden.
</think>
TNI dipegang kekuasaan oleh Presiden sebagai pimpinan tertinggi.
<｜end▁of▁sentence｜>


## Evaluasi ROUGE atau BLEU 

## Save Model

In [17]:
# Setelah training selesai
trainer_lawbot.save_model("model_lora_lawbot")
tokenizer.save_pretrained("model_lora_lawbot") 

('model_lora_lawbot/tokenizer_config.json',
 'model_lora_lawbot/special_tokens_map.json',
 'model_lora_lawbot/tokenizer.json')

In [18]:
!zip -r model_lora_lawbot.zip model_lora_lawbot

  adding: model_lora_lawbot/ (stored 0%)
  adding: model_lora_lawbot/adapter_config.json (deflated 56%)
  adding: model_lora_lawbot/special_tokens_map.json (deflated 69%)
  adding: model_lora_lawbot/tokenizer_config.json (deflated 95%)
  adding: model_lora_lawbot/tokenizer.json (deflated 85%)
  adding: model_lora_lawbot/training_args.bin (deflated 53%)
  adding: model_lora_lawbot/README.md (deflated 66%)
  adding: model_lora_lawbot/adapter_model.safetensors (deflated 7%)


In [19]:
from IPython.display import FileLink

# Membuat link untuk download
FileLink("model_lora_lawbot.zip")

/kaggle/working/model_lora_lawbot.zip

## Testing

In [20]:
from unsloth import FastLanguageModel
from peft import PeftModel
from transformers import AutoTokenizer

import torch

max_seq_length = 2048
dtype = None
load_in_4bit = True

# Path ke folder model yang sudah disimpan
model_dir = "/kaggle/working/model_lora_lawbot"

# 1. Load tokenizer dari folder hasil fine-tuning
tokenizer = AutoTokenizer.from_pretrained(model_dir, use_fast=True)

# 2. Load base model (harus sama dengan yang dipakai saat fine-tuning)
model, _ = FastLanguageModel.from_pretrained(
    model_name="unsloth/DeepSeek-R1-Distill-Llama-8B",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    token=hugging_face_token,  # <- pastikan ini sudah Anda set sebelumnya
)

# 3. Load LoRA weights yang sudah difinetune
model_lora_capd = PeftModel.from_pretrained(model, model_dir)
model_lora_capd.eval()

# 4. Prompt satu pertanyaan CAPD
prompt = """Di bawah ini adalah instruksi yang menjelaskan tugas, dipasangkan dengan input yang memberikan konteks lebih lanjut.
Tuliskan respons yang menyelesaikan permintaan dengan tepat.
Sebelum menjawab, pikirkan dengan cermat pertanyaan tersebut dan buatlah rangkaian pemikiran langkah demi langkah untuk memastikan respons yang logis dan akurat.

### Instruksi:
Anda adalah seorang ahli hukum dengan pengetahuan tingkat lanjut dalam penalaran hukum, analisis kasus, dan penyusunan dokumen hukum. 
Jawablah pertanyaan hukum berikut ini dengan tepat, berdasarkan peraturan perundang-undangan yang berlaku dan preseden hukum yang relevan.

### Pertanyaan:
Apa arti dari “berada di bawah Presiden” dalam konteks TNI?

### Jawaban:
<think>
"""

# 5. Tokenisasi & inference
inputs = tokenizer(prompt, return_tensors="pt").to(model_lora_capd.device)

with torch.inference_mode():
    outputs = model_lora_lawbot.generate(
        **inputs,
        max_new_tokens=1200,
        do_sample=False,
        temperature=0.7,
        top_p=0.95,
        eos_token_id=tokenizer.eos_token_id
    )

# 6. Decode dan tampilkan jawaban
decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("\n==== Jawaban Model ====\n")
print(decoded)

==((====))==  Unsloth 2025.5.7: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla P100-PCIE-16GB. Num GPUs = 1. Max memory: 15.888 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 6.0. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!

==== Jawaban Model ====

Di bawah ini adalah instruksi yang menjelaskan tugas, dipasangkan dengan input yang memberikan konteks lebih lanjut.
Tuliskan respons yang menyelesaikan permintaan dengan tepat.
Sebelum menjawab, pikirkan dengan cermat pertanyaan tersebut dan buatlah rangkaian pemikiran langkah demi langkah untuk memastikan respons yang logis dan akurat.

### Instruksi:
Anda adalah seorang ahli hukum dengan pengetahuan tingkat lanjut dalam penalaran hukum, analisis kasus, dan penyusunan dokumen hukum. 
Jawablah pertanyaan h